In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("/kaggle/input/playground-train/train.csv")
data.head()

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.0
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.9
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.0


In [3]:
data.dtypes

id                    int64
age                   int64
gender               object
course               object
study_hours         float64
class_attendance    float64
internet_access      object
sleep_hours         float64
sleep_quality        object
study_method         object
facility_rating      object
exam_difficulty      object
exam_score          float64
dtype: object

In [4]:
col = data.columns
col

Index(['id', 'age', 'gender', 'course', 'study_hours', 'class_attendance',
       'internet_access', 'sleep_hours', 'sleep_quality', 'study_method',
       'facility_rating', 'exam_difficulty', 'exam_score'],
      dtype='object')

In [5]:
def missing(col):
    mis = {}
    for i in col:
        mis[i] = data[i].isna().sum()
    return mis

In [6]:
print(missing(col))

{'id': np.int64(0), 'age': np.int64(0), 'gender': np.int64(0), 'course': np.int64(0), 'study_hours': np.int64(0), 'class_attendance': np.int64(0), 'internet_access': np.int64(0), 'sleep_hours': np.int64(0), 'sleep_quality': np.int64(0), 'study_method': np.int64(0), 'facility_rating': np.int64(0), 'exam_difficulty': np.int64(0), 'exam_score': np.int64(0)}


In [7]:
def obj(col):
    ob = []
    for i in col:
        if data[i].dtype == "object":
            ob.append(i)
        else:
            pass
    return ob

In [8]:
o1 = obj(col)
o1

['gender',
 'course',
 'internet_access',
 'sleep_quality',
 'study_method',
 'facility_rating',
 'exam_difficulty']

In [9]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [10]:
one = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
Transform = ColumnTransformer(transformers = [("one", one, o1)] , remainder = "passthrough")

In [11]:
y = data["exam_score"]
x = data.drop(columns=["exam_score"])

In [12]:
len(x) , len(y)

(630000, 630000)

In [13]:
t_x = Transform.fit_transform(x)

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train , x_test , y_train , y_test =  train_test_split(t_x , y , test_size = 0.2)

In [16]:
len(x_train) , len(x_test) , len(y_train) , len(y_test)

(504000, 126000, 504000, 126000)

In [17]:
from lightgbm import LGBMRegressor
m1 = LGBMRegressor(objective = "regression" , random_state = 1)

In [18]:
grid = {"n_estimators": [500, 1000, 2000],
    "learning_rate": [0.05, 0.03, 0.01],
    "num_leaves": [31, 63, 127],
    "max_depth": [-1, 10, 20],
    "min_child_samples": [20, 40, 60],
    "subsample": [0.7, 0.8, 0.9],
    "colsample_bytree": [0.7, 0.8, 0.9],
    "reg_alpha": [0.0, 0.1, 0.5],
    "reg_lambda": [0.0, 0.1, 0.5]}

In [19]:
from sklearn.model_selection import KFold

In [20]:
from sklearn.model_selection import RandomizedSearchCV

best = RandomizedSearchCV(
    estimator=m1,
    param_distributions=grid,
    n_iter=20,                      
    scoring="neg_root_mean_squared_error",
    cv=3,
    n_jobs=-1,
    random_state=42,
    verbose=2
)

best.fit(x_train, y_train)


Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/v

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.089730 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 876
[LightGBM] [Info] Number of data points in the train set: 336000, number of used features: 31
[LightGBM] [Info] Start training from score 62.517523
[CV] END colsample_bytree=0.7, learning_rate=0.05, max_depth=10, min_child_samples=20, n_estimators=1000, num_leaves=63, reg_alpha=0.5, reg_lambda=0.1, subsample=0.9; total time= 1.6min
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.117911 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 876
[LightGBM] [Info] Number of data points in the train set: 336000, number of used features: 31
[LightGBM] [Info] Start training from score 62.535919
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=10, min_child_samples=40, n_estimators=500, num_leaves=63, reg_alph

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.127960 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 876
[LightGBM] [Info] Number of data points in the train set: 336000, number of used features: 31
[LightGBM] [Info] Start training from score 62.535919
[CV] END colsample_bytree=0.9, learning_rate=0.03, max_depth=-1, min_child_samples=60, n_estimators=500, num_leaves=31, reg_alpha=0.0, reg_lambda=0.0, subsample=0.7; total time= 1.1min
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.101724 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 876
[LightGBM] [Info] Number of data points in the train set: 336000, number of used features: 31
[LightGBM] [Info] Start training from score 62.524725
[CV] END colsample_bytree=0.7, learning_rate=0.05, max_depth=10, min_child_samples=20, n_estimators=1000, num_leaves=63, reg_alph

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.053515 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 876
[LightGBM] [Info] Number of data points in the train set: 336000, number of used features: 31
[LightGBM] [Info] Start training from score 62.517523
[CV] END colsample_bytree=0.9, learning_rate=0.03, max_depth=-1, min_child_samples=60, n_estimators=500, num_leaves=31, reg_alpha=0.0, reg_lambda=0.0, subsample=0.7; total time= 1.0min
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.057638 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 876
[LightGBM] [Info] Number of data points in the train set: 336000, number of used features: 31
[LightGBM] [Info] Start training from score 62.535919
[

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020086 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 876
[LightGBM] [Info] Number of data points in the train set: 504000, number of used features: 31
[LightGBM] [Info] Start training from score 62.526056


RandomizedSearchCV(cv=3,
                   estimator=LGBMRegressor(objective='regression',
                                           random_state=1),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.7, 0.8, 0.9],
                                        'learning_rate': [0.05, 0.03, 0.01],
                                        'max_depth': [-1, 10, 20],
                                        'min_child_samples': [20, 40, 60],
                                        'n_estimators': [500, 1000, 2000],
                                        'num_leaves': [31, 63, 127],
                                        'reg_alpha': [0.0, 0.1, 0.5],
                                        'reg_lambda': [0.0, 0.1, 0.5],
                                        'subsample': [0.7, 0.8, 0.9]},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=2)

In [36]:
from sklearn.metrics import mean_squared_error
y_val_pred = best.predict(x_test)
rmse = mean_squared_error(y_test, y_val_pred)
rmse = rmse ** 0.5
print(rmse)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


8.767070801399814


In [37]:
test = pd.read_csv("/kaggle/input/playground-test/test.csv")

In [38]:
test

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty
0,630000,24,other,ba,6.85,65.2,yes,5.2,poor,group study,high,easy
1,630001,18,male,diploma,6.61,45.0,no,9.3,poor,coaching,low,easy
2,630002,24,female,b.tech,6.60,98.5,yes,6.2,good,group study,medium,moderate
3,630003,24,male,diploma,3.03,66.3,yes,5.7,average,mixed,medium,moderate
4,630004,20,female,b.tech,2.03,42.4,yes,9.2,average,coaching,low,moderate
...,...,...,...,...,...,...,...,...,...,...,...,...
269995,899995,21,other,b.com,2.55,82.3,yes,8.4,average,mixed,medium,hard
269996,899996,17,female,b.com,0.49,46.4,yes,8.8,good,mixed,low,easy
269997,899997,22,male,bba,6.62,74.7,yes,5.5,good,coaching,high,easy
269998,899998,22,other,ba,4.08,51.8,yes,8.7,poor,online videos,high,moderate


In [39]:
test.columns

Index(['id', 'age', 'gender', 'course', 'study_hours', 'class_attendance',
       'internet_access', 'sleep_hours', 'sleep_quality', 'study_method',
       'facility_rating', 'exam_difficulty'],
      dtype='object')

In [40]:
t_test = Transform.fit_transform(test)
preds = best.predict(t_test)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
preds

In [ ]:
submission = pd.DataFrame({"id": test["id"], "exam_score": preds}) 

In [ ]:
submission.to_csv("2nd_submission.csv")